In [182]:
import pandas as pd
import jieba
from gensim.models import word2vec
import numpy as np
#导入Keras中的卷积工具
from keras.models import Sequential  #基础的Keras神经网络模型
from keras.layers import Dense, Dropout, Activation #模型中常用的层对象
from keras.layers import Conv1D,GlobalMaxPooling1D #卷积层和池化

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import  Input, Dense,Embedding,Conv1D,MaxPooling1D,LSTM
from keras.models import Model
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.callbacks import TensorBoard,EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import load_model


df = pd.read_csv('./data.csv')
df=df[['text','location', 'country']]

#print("在 text 列中总共有 %d 个空值." % df['text'].isnull().sum())
#print("在 location 列中总共有 %d 个空值." % df['location'].isnull().sum())
df[df.isnull().values==True]
df = df[pd.notnull(df['text'])]
#print(df)

# df.sample(2)
texts = [[word for word in jieba.cut(document)] for document in df['text']]

print(texts)
# 标签编码
df['country_id'] = df['country'].factorize()[0]
df['location_id'] = df['location'].factorize()[0]
cat_id_df = df[['country', 'country_id']].drop_duplicates().sort_values('country_id').reset_index(drop=True)
cat_id_df = df[['location', 'location_id']].drop_duplicates().sort_values('location_id').reset_index(drop=True)
print(df)


[['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['美韩', '在', '日本海', '举行', '联合', '军演'], ['中俄于', '2023', '年', '9', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中泰', '在', '南海', '举行', '联合', '军演'], ['中', '缅', '在', '珊瑚', '海', '举行', '联合', '军演'], ['美菲于', '2008', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['日', '韩', '在', '日本海', '举行', '联合', '军演'], ['中老于', '2013', '年', '2', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中', '柬', '在', '南海', '举行', '联合', '军演'], ['新加坡', '在', '吕宋岛', '举行', '军演'], ['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['美韩', '在', '日本海', '举行', '联合', '军演'], ['中俄于', '2023', '年', '9', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中泰', '在', '南海', '举行', '联合', '军演'], ['中', '缅', '在', '珊瑚', '海', '举行', '联合', '军演'], ['美菲于', '2008', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['日', '韩', '在', '日本海', '举行', '联合', '军演'], ['中老于', '2013', '年', '2', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中', '柬', '在', '南海', '

In [183]:
X_train, X_test, Y_train, Y_test = train_test_split(texts,df['location_id'],test_size=0.2, random_state=42)
print(X_train, X_test, Y_train, Y_test)

[['中', '柬', '在', '南海', '举行', '联合', '军演'], ['美菲于', '2008', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['美韩', '在', '日本海', '举行', '联合', '军演'], ['中泰', '在', '南海', '举行', '联合', '军演'], ['中', '柬', '在', '南海', '举行', '联合', '军演'], ['日', '韩', '在', '日本海', '举行', '联合', '军演'], ['中泰', '在', '南海', '举行', '联合', '军演'], ['中俄于', '2023', '年', '9', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['新加坡', '在', '吕宋岛', '举行', '军演'], ['新加坡', '在', '吕宋岛', '举行', '军演'], ['中', '缅', '在', '珊瑚', '海', '举行', '联合', '军演'], ['中俄于', '2023', '年', '9', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中老于', '2013', '年', '2', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['中', '缅', '在', '珊瑚', '海', '举行', '联合', '军演'], ['日', '韩', '在', '日本海', '举行', '联合', '军演']] [['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['中老于', '2013', '年', '2', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['美菲于', '2008', '年', '9', '月', '10', '日', '在', '黄海', '举行',

In [195]:
max_words = 100
max_len = 100
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(texts)

train_seq = tok.texts_to_sequences(X_train)
test_seq = tok.texts_to_sequences(X_test)

train_seq_mat = pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = pad_sequences(test_seq,maxlen=max_len)

# 标签独热编码
ohe = OneHotEncoder()
y_train_seq = ohe.fit_transform(np.array(Y_train).reshape(-1,1)).toarray()
y_test_seq = ohe.transform(np.array(Y_test).reshape(-1,1)).toarray()


print(texts)
print(train_seq)
print(train_seq_mat)
print('test_seq', test_seq)
print('test_seq_mat', test_seq_mat)
print('y_train_seq', y_train_seq)
print('y_test_seq', y_test_seq)

[['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['美韩', '在', '日本海', '举行', '联合', '军演'], ['中俄于', '2023', '年', '9', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中泰', '在', '南海', '举行', '联合', '军演'], ['中', '缅', '在', '珊瑚', '海', '举行', '联合', '军演'], ['美菲于', '2008', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['日', '韩', '在', '日本海', '举行', '联合', '军演'], ['中老于', '2013', '年', '2', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中', '柬', '在', '南海', '举行', '联合', '军演'], ['新加坡', '在', '吕宋岛', '举行', '军演'], ['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['美韩', '在', '日本海', '举行', '联合', '军演'], ['中俄于', '2023', '年', '9', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中泰', '在', '南海', '举行', '联合', '军演'], ['中', '缅', '在', '珊瑚', '海', '举行', '联合', '军演'], ['美菲于', '2008', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演'], ['日', '韩', '在', '日本海', '举行', '联合', '军演'], ['中老于', '2013', '年', '2', '月', '10', '日', '在', '东海', '举行', '联合', '军演'], ['中', '柬', '在', '南海', '

In [196]:

num_classes = 6
## 定义CNN-LSTM模型
inputs = Input(name='inputs',shape=[max_len])
## Embedding(词汇表大小,batch大小,每个新闻的词长)
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(layer)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(layer)
layer = MaxPooling1D(pool_size=2)(layer)
layer = LSTM(100, dropout=0.2, recurrent_dropout=0.2)(layer)
layer = Dense(num_classes, activation='softmax')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_seq_mat,y_train_seq,batch_size=128,epochs=10,
                      validation_data=(test_seq_mat,y_test_seq),
                      callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.000001),TensorBoard(log_dir='./log')]
                        ## 当val-loss不再降低时停止训练
                     )
# # 保存模型
model.save('CNN-LSTM-10.h5')
del model

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 100)]             0         
                                                                 
 embedding_16 (Embedding)    (None, 100, 128)          12928     
                                                                 
 conv1d_62 (Conv1D)          (None, 100, 32)           12320     
                                                                 
 max_pooling1d_32 (MaxPooli  (None, 50, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_63 (Conv1D)          (None, 50, 32)            3104      
                                                                 
 max_pooling1d_33 (MaxPooli  (None, 25, 32)            0         
 ng1D)                                                    

/Users/bierxiensi/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [197]:
predict_df = pd.read_csv('./predict.csv')
predict_texts = [[word for word in jieba.cut(document)] for document in predict_df['text']]
predict_seq = tok.texts_to_sequences(predict_texts)
predict_seq_mat = pad_sequences(predict_seq,maxlen=max_len)
print('predict_texts', predict_texts)
print('predict_seq', predict_seq)
print('predict_seq_mat', predict_seq_mat)

predict_texts [['美日于', '2023', '年', '9', '月', '10', '日', '在', '黄海', '举行', '联合', '军演', '，', '台湾', '参与', '其中'], ['中老', '在', '东海', '举行', '联合', '军演'], ['中', '柬', '在', '南海', '举行', '联合', '军演'], ['新加坡', '在', '吕宋岛', '举行', '军演']]
predict_seq [[16, 10, 6, 9, 7, 8, 5, 1, 11, 2, 4, 3], [1, 13, 2, 4, 3], [15, 29, 1, 14, 2, 4, 3], [30, 1, 31, 2, 3]]
predict_seq_mat [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 16 10  6  9  7  8  5  1
  11  2  4  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
  13  2  4  3]
 [ 0  0  0  0  0  0  0  

In [208]:
# 导入已经训练好的模型
model = load_model('CNN-LSTM-10.h5')
predict_pre = model.predict(predict_seq_mat)
pred = np.argmax(predict_pre, axis=1)

print(pred, predict_pre, predict_df['text'])
labels = ['黄海', '日本海', '东海', '南海', '吕宋岛']
count=0
for i in pred:
    print('待分类文本：', predict_df['text'][count])
    print('地点分类结果：', labels[i])
    count = count + 1


1/1 [==============================] - 0s 92ms/step
[2 3 3 3] [[0.16508111 0.16590548 0.17152835 0.1705709  0.16230057 0.16461352]
 [0.16453905 0.16629338 0.1699449  0.1714949  0.16261694 0.16511084]
 [0.16339193 0.16595112 0.16931581 0.17282142 0.16349532 0.16502431]
 [0.16411512 0.16633752 0.16880742 0.1722457  0.16309933 0.16539495]] 0    美日于2023年9月10日在黄海举行联合军演，台湾参与其中
1                      中老在东海举行联合军演
2                      中柬在南海举行联合军演
3                      新加坡在吕宋岛举行军演
Name: text, dtype: object
待分类文本： 美日于2023年9月10日在黄海举行联合军演，台湾参与其中
地点分类结果： 东海
待分类文本： 中老在东海举行联合军演
地点分类结果： 南海
待分类文本： 中柬在南海举行联合军演
地点分类结果： 南海
待分类文本： 新加坡在吕宋岛举行军演
地点分类结果： 南海
